## 인터파크 - 베스트셀러

In [1]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup

- Step 1) 사이트에 접속하고 HTML 데이터를 가져와 파싱

In [2]:
url = 'http://book.interpark.com/display/collectlist.do?_method=bestsellerHourNew&bookblockname=b_gnb&booklinkname=%BA%A3%BD%BA%C6%AE%C1%B8&bid1=w_bgnb&bid2=LiveRanking&bid3=main&bid4=001'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [3]:
print(res.text[:1000])




<!DOCTYPE html>
<html lang="ko">
<head>

 

    <title>인터파크 도서</title>
    <meta http-equiv="Content-Type" content="text/html; charset=euc-kr">
    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
    <meta name="keywords" content="인터파크도서, 인터넷 서점, eBook, 비스킷탭, 음반, DVD, 중고책, 중고책서점, 신간도서, 국내도서, 외국도서, 추천도서">
    <meta name="description" content="문학은 인터파크 도서!">
    <meta http-equiv="Cache-Control" content="no-cache"/>
    <meta http-equiv="Expires" content="0"/>
    <meta http-equiv="Pragma" content="no-cache"/>
    <link rel="shortcut icon" href="http://qi-b.qoo10cdn.com/interpark.ico">
    <meta http-equiv="imagetoolbar" content="no"/>
    <meta name="naver-site-verification" content="295287bf1753708aabe3586a6cbb4b39761ff03b" />
    <script type="text/javascript">
        var isFancyAlert = false;    // IMFS에서는 msgAlert 기능을 비활성화시킴
        var imfsSSLDomain = "https://book.interpark.com"; // com.interpark.util.web.ServletUtils.IMFS_URL_SSL 값을 사용하였으나,


- Step 2) 원하는 데이터 찾기

In [4]:
lis = soup.select('.rankBestContentList > ol > li')
len(lis)

15

- Step 3) 하나를 선택해서 원하는 정보 추출

In [5]:
li = lis[6]

In [6]:
# 책 제목
title = li.select_one('.itemName').get_text().strip()
title

'가짜 노동'

In [7]:
# 저자
author = li.select_one('.author').get_text().strip()
author

'데니스 뇌르마르크, 아네르스 포그 옌센/이수영 역'

In [8]:
# 출판사
company = li.select_one('.company').get_text().strip()
company

'자음과모음'

In [11]:
# 가격
price = li.select_one('.price > em').get_text().strip()
price = int(price.replace(',', ''))
price

15120

In [ ]:
# 순위
"""
<div class="rankNumber digit2">
    <span class="rankBtn_ctrl rkNum_B06 "></span>
</div>

<div class="rankNumber digit2">
    <span class="rankBtn_ctrl rkNum_M01 "></span>
    <span class="rankBtn_ctrl rkNum_M00 "></span>
</div>
"""

In [18]:
rank_data = lis[6].select('.rankBtn_ctrl')
rank_data[0]['class'] 


['rankBtn_ctrl', 'rkNum_B06']

In [20]:
rank_data = lis[10].select('.rankBtn_ctrl')
rank_data[1]['class']

['rankBtn_ctrl', 'rkNum_M00']

In [21]:
if len(rank_data) == 1:
    rank = rank_data[0]['class'][1][-1]    
else:
    rank = rank_data[0]['class'][1][-1] + rank_data[1]['class'][1][-1]
rank

'10'

- Step 4) 반복문을 사용하여 페이지 내에 있는 데이터 가져오기

In [22]:
data = []
for li in lis:
    rank_data = li.select('.rankBtn_ctrl')
    if len(rank_data) == 1:
        rank = int(rank_data[0]['class'][1][-1])
    else:
        rank = int(rank_data[0]['class'][1][-1] + rank_data[1]['class'][1][-1])
    title = li.select_one('.itemName').get_text().strip()
    author = li.select_one('.author').get_text().strip()
    company = li.select_one('.company').get_text().strip()
    price = li.select_one('.price > em').get_text().strip()
    price = int(price.replace(',', ''))
    data.append({'순위':rank, '제목':title, '저자':author, '출판사':company, '가격':price})

- Step 5) 오류가 발생하면 오류를 해결하고 2, 3, 4 과정을 반복

- Step 6) 모든 페이지에 대해 반복하기

- Step 7) 데이터프레임으로 만들기

In [23]:
df = pd.DataFrame(data)
df

,순위,제목,저자,출판사,가격
0,1,디케의 눈물,조국,다산북스,16920
1,2,도시와 그 불확실한 벽,무라카미 하루키(村上春樹),문학동네,17550
2,3,퓨처 셀프,벤저민 하디/최은아 역,상상스퀘어,17820
3,4,1%를 읽는 힘,메르,토네이도,19800
4,5,흔한남매 14,"흔한남매, 백난도/유난희 그림/흔한컴퍼니 감수",미래엔아이세움,13050
5,6,일론 머스크,월터 아이작슨,21세기북스,34200
6,6,가짜 노동,"데니스 뇌르마르크, 아네르스 포그 옌센/이수영 역",자음과모음,15120
7,8,세이노의 가르침,세이노,데이원,6480
8,9,트렌드 코리아 2024,"김난도, 추예린, 전다현, 전미영, 최지혜",미래의창,17100
9,10,완전배출,조승우,사이몬북스,17100
